# zad 3 Jakub Iliński

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Tuple
import torch
import torch.nn.functional as F 

PAPUGA = 'flax-community/papuGaPT2'
DEVICE = "cuda"

tokenizer = AutoTokenizer.from_pretrained(PAPUGA)
model = AutoModelForCausalLM.from_pretrained(PAPUGA).to(DEVICE)
model.device

device(type='cuda', index=0)

In [23]:
POSITIVE_SENTENCES: List[str] = [
    "Parking monitorowany w cenie.",
    "Hotel czysty, pokoje były sprzątane bardzo dokłądnie.",
    "Generalnie mogę go polecić, kierował mnie na potrzebne badania, analizował ich wyniki, cierpliwie odpowiadał na pytania.",
    "Fajny klimat pofabrykanckich kamienic.",
    "Sala zabaw dla dzieci, plac zabaw na zewnątrz, kominek, tenis stołowy.",
]

NEGATIVE_SENTENCES: List[str] = [
    "W wielu pokojach niedziałająca klimatyzacja.",
    "Jedzenie mimo rzekomych dni europejskich monotonne.",
    "Drożej niż u konkurencji w podobnym standardzie.",
    "Może za szybko zrezygnowałam, ale szkoda mi było wydawać pieniędzy na spotkania, które nie przynosiły efektu.",
    "Omijaj to miejsce!",
]

In [24]:
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def get_prob(context: str, label: str) -> float:
    full = context + label
    
    input_ids_full = tokenizer(full, return_tensors='pt')['input_ids'].to(DEVICE)
    input_ids_context = tokenizer(context, return_tensors='pt')['input_ids'].to(DEVICE)
    
    with torch.no_grad():
        output = model(input_ids=input_ids_full)
        logits, labels = output.logits[:, :-1, :], input_ids_full[:, 1:]
        log_probs = log_probs_from_logits(logits, labels)
        
    context_len, full_len = input_ids_context.size(1), input_ids_full.size(1)
    start, end = max(context_len - 1, 0), full_len - 1
    
    label_log_probs = log_probs[:, start:end]
    return float(label_log_probs.mean().item())

In [25]:
from abc import ABC, abstractmethod

class Template(ABC):
    context: str
    positive_label: str
    negative_label: str
    
    @abstractmethod
    def __init__(self, text: str) -> None: ...
    
class TemplateA(Template):
    def __init__(self, text: str) -> None:
        self.context = f"Opinia: {text}\nSentyment: "
        self.positive_label = "pozytywny."
        self.negative_label = "negatywny."
        
class TemplateB(Template):
    def __init__(self, text: str) -> None:
        self.context = f"{text}, Czy ta opinia jest pozytywna: "
        self.positive_label = "tak."
        self.negative_label = "nie."
        
class TemplateC(Template):
    def __init__(self, text: str) -> None:
        self.context = f"{text}, Czy ta opinia ma sentyment: "
        self.positive_label = "pozytywny."
        self.negative_label = "negatywny."

In [26]:
def predict_sentiment(text: str, temp: type[Template]) -> Tuple[str, float, float]:
    t = temp(text)
    pos_prob = get_prob(t.context, t.positive_label)
    neg_prob = get_prob(t.context, t.negative_label)
    sentiment = "GOOD" if pos_prob > neg_prob else "BAD"
    return (sentiment, pos_prob, neg_prob)

def evaluate(samples: List[str], temp: type[Template]) -> float:
    hits = 0
    for sample in samples: 
        if sample[0] == "G":
            label, text = sample[:4], sample[4:] 
        else:
            label, text = sample[:3], sample[3:]
            
        out, pos_prob, neg_prob = predict_sentiment(text, temp)
        if out == label:
            hits += 1
    return hits / len(samples)

def load_reviews() -> List[str]:
    with open("reviews_for_task3.txt", "r") as file:
        return [line.rstrip("\n") for line in file if line.strip()]

In [27]:
quick_test = [f"GOOD {s}" for s in POSITIVE_SENTENCES] + [f"BAD {s}" for s in NEGATIVE_SENTENCES]

acc_A = evaluate(quick_test, TemplateA)
acc_B = evaluate(quick_test, TemplateB)
acc_C = evaluate(quick_test, TemplateC)

print(f"A: {acc_A:.2%} B: {acc_B:.2%} C: {acc_C:.2%}")

A: 50.00% B: 50.00% C: 70.00%


In [28]:
test = load_reviews()

acc_A = evaluate(test, TemplateA)
acc_B = evaluate(test, TemplateB)
acc_C = evaluate(test, TemplateC)

print(f"A: {acc_A:.2%} B: {acc_B:.2%} C: {acc_C:.2%}")

A: 56.00% B: 52.50% C: 66.25%
